In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from numpy.linalg import inv, eigh

# ---------- 1) Load & clean ----------
PATH = "HDFS_structured.csv"

df = pd.read_csv(PATH)

In [2]:
# Display the first 5 rows
print(df.head())

   LineId  Label              Timestamp                   BlockId  \
0       1      0              Timestamp                       NaN   
1       2      0  "2008-11-09 20:35:18"  blk_-1608999687919862906   
2       3      0  "2008-11-09 20:35:18"  blk_-1608999687919862906   
3       4      0  "2008-11-09 20:35:19"  blk_-1608999687919862906   
4       5      0  "2008-11-09 20:35:19"  blk_-1608999687919862906   

                                             Content  \
0                                    BlockId Message   
1  "dfs.DataNode$DataXceiver: Receiving block blk...   
2  "dfs.FSNamesystem: BLOCK* NameSystem.allocateB...   
3  "dfs.DataNode$DataXceiver: Receiving block blk...   
4  "dfs.DataNode$DataXceiver: Receiving block blk...   

                                       EventTemplate   EventId  
0                                    BlockId Message  1ea271b9  
1  <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*...  e8ffe641  
2  "dfs.FSNamesystem: BLOCK* NameSystem.allocateB... 

In [3]:
# Drop the spurious header-like row (where Timestamp literally equals "Timestamp" or BlockId is NaN)
mask_junk = (df['Timestamp'].astype(str).str.strip('"') == 'Timestamp') | df['BlockId'].isna()
df = df.loc[~mask_junk].copy()

# Normalize columns
# --- Normalize block id column safely ---
if 'block_id' not in df.columns and 'BlockId' in df.columns:
    df['block_id'] = df['BlockId']
elif 'block_id' in df.columns and df['block_id'].isna().all() and 'BlockId' in df.columns:
    df['block_id'] = df['BlockId']
# if both exist and are valid, keep as is

# Drop rows missing block_id or EventId
df = df.dropna(subset=['block_id', 'EventId'])

# Parse Timestamp like "2008-11-09 20:35:18"
df['Timestamp'] = pd.to_datetime(df['Timestamp'].astype(str).str.strip('"'), errors='coerce')
# If some rows didn't parse, keep them but they'll get hour 0 (goes to 'night')
df['hour'] = df['Timestamp'].dt.hour.fillna(0).astype(int)
df['hour_bin'] = pd.cut(df['hour'], bins=[-1,5,11,17,23],
                        labels=['night','morning','afternoon','evening']).astype(str)

# ---------- 2) Build session-event matrix X ----------
# Count of EventId per block (use presence/absence for Jaccard)
event_counts = (
    df.groupby(['block_id', 'EventId'])
      .size()
      .unstack(fill_value=0)
      .astype(np.int32)
)
X_counts = event_counts
X_bin = (X_counts > 0).astype(int)

# ---------- 3) Build predictors Y (label + hour_bin) ----------
# Label may already be 0/1 in this file; if it's strings, map to 0/1
# We'll take the first label per block_id (HDFS labels are per-session)
lab_per_block = (
    df.groupby('block_id')['Label'].first()
      .pipe(lambda s: s.astype(str).str.lower().map({'normal':0,'anomaly':1})
            if s.dtype == 'O' or s.dtype.name.startswith('str') else s.astype(int))
)

hourbin_per_block = df.groupby('block_id')['hour_bin'].first().astype(str)

Y_df = pd.DataFrame({
    'label': lab_per_block.reindex(X_counts.index).fillna(0).astype(int),
    'hour_bin': hourbin_per_block.reindex(X_counts.index).fillna('unknown').astype(str),
})
Y = pd.get_dummies(Y_df, drop_first=True).astype(float)  # one-hot for hour_bin

In [6]:
# ---------- 4) Distances (non-parametric) with stratified sampling ----------
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd

# Choose a manageable cap for sessions (adjust to your RAM; 4000–8000 is typical)
MAX_N = 6000
rng = np.random.default_rng(42)

# We'll stratify by the block-level label you already built (Y_df['label'])
block_labels = Y_df['label'].reindex(X_bin.index)

# Compute per-class quotas
counts = block_labels.value_counts(dropna=False)
total = counts.sum()
quotas = (counts / total * MAX_N).round().astype(int).clip(lower=1)

# Sample indices per class
sample_idx = []
for cls, q in quotas.items():
    idx_cls = block_labels[block_labels == cls].index
    q = min(q, len(idx_cls))
    # random sample without replacement
    pick = rng.choice(idx_cls.to_numpy(), size=q, replace=False)
    sample_idx.append(pick)
sample_idx = np.concatenate(sample_idx)

# Subset X and Y consistently
X_bin_s = X_bin.loc[sample_idx]
Y_s = Y.loc[sample_idx]

# Convert to boolean for efficient Jaccard in SciPy
X_bool = X_bin_s.values.astype(bool)

# Condensed vector -> square matrix; cast to float32 to save memory
D = squareform(pdist(X_bool, metric='jaccard')).astype(np.float32)

print(f"Using {len(sample_idx)} sessions; distance matrix shape: {D.shape}, dtype: {D.dtype}")


Using 6000 sessions; distance matrix shape: (6000, 6000), dtype: float32


In [7]:
# ---------- 5) db-RDA (CAP) ----------
def gower_center(D):
    D2 = D**2
    n = D.shape[0]
    J = np.eye(n) - np.ones((n, n))/n
    return -0.5 * J @ D2 @ J

def hat_matrix(Y):
    XtX = Y.T @ Y
    return Y @ inv(XtX) @ Y.T

def dbrda(D, Y_df, add_intercept=True, n_perm=999, random_state=42):
    rng = np.random.default_rng(random_state)
    n = D.shape[0]
    G = gower_center(D)

    # Scale continuous columns only (heuristic: non-binary)
    Yc = Y_df.copy()
    non_binary = [c for c in Yc.columns if not set(Yc[c].unique()).issubset({0.0,1.0})]
    if non_binary:
        Yc[non_binary] = StandardScaler().fit_transform(Yc[non_binary])

    Ymat = Yc.values
    if add_intercept:
        Ymat = np.column_stack([np.ones(n), Ymat])

    H = hat_matrix(Ymat)
    num = np.trace(H @ G @ H)
    den = np.trace(G)
    R2 = num / den

    # Permutation p-value (shuffle rows of Y)
    greater = 0
    for _ in range(n_perm):
        perm = rng.permutation(n)
        Hp = hat_matrix(Ymat[perm, :])
        R2p = np.trace(Hp @ G @ Hp) / den
        if R2p >= R2:
            greater += 1
    pval = (greater + 1) / (n_perm + 1)

    # Canonical axes (for plotting if you like)
    HGH = H @ G @ H
    eigvals, eigvecs = eigh(HGH)
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    scores2d = eigvecs[:, :2] * np.sqrt(np.maximum(eigvals[:2], 0))

    return {"R2": float(R2), "pval": float(pval), "scores2d": scores2d, "eigvals": eigvals}

# res = dbrda(D, Y_s, add_intercept=True, n_perm=999, random_state=7)
# print(f"db-RDA on sampled HDFS logs → R^2={res['R2']:.4f}, p={res['pval']:.4f}")


In [8]:
def partial_dbrda(D, Y_full, target_col, n_perm=999, random_state=42):
    """Unique effect of one predictor, controlling for all others."""
    others = Y_full.drop(columns=[target_col])
    res_all = dbrda(D, Y_full, n_perm=199, random_state=random_state)
    res_others = dbrda(D, others, n_perm=199, random_state=random_state)
    unique_r2 = res_all['R2'] - res_others['R2']
    return unique_r2

for col in Y_s.columns:
    uniq = partial_dbrda(D, Y_s, col)
    print(f"Unique contribution of {col:<20} = {uniq:.4f}")

Unique contribution of label                = 0.0162
Unique contribution of hour_bin_evening     = 0.1017
Unique contribution of hour_bin_morning     = 0.0257
Unique contribution of hour_bin_night       = 0.0886
